In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm, ttest_ind

## Формула для оценки размера выборки

$$n > \dfrac{\left[ \Phi^{-1} \Bigl( 1-\dfrac{\alpha}{2} \Bigr) + \Phi^{-1} \Bigl( 1-\beta \Bigr) \right]^2 (\sigma_X^2 + \sigma_Y^2)}{\varepsilon^2}$$

$\alpha$ - вероятность ошибки I рода, он же уровень значимости - договорились, что фиксируем на уровне 5% (0.05)

$\beta$ - вероятность ошибки II рода, с какой вероятностью мы скажем, что эффекта нет, когда он на самом деле есть - договорились, что фиксируем на уровне 20% (0.2)

$\varepsilon$ - минимальный детектируемый эффект - MDE

$\sigma{^2}$ - дисперсия в оригинале и вариации, для упрощения возьмем, что дисперсии равны, тогда вместо $(\sigma_X^2 + \sigma_Y^2)$ возьмём 2$\sigma{^2}$


$\Phi^{-1}$ - обратная функция нормального распределения
- При $\alpha$ = 0.05 получаем $\Phi^{-1}(1-\dfrac{\alpha}{2})$ = 1.96
- При $\beta$ = 0.2 получаем $\Phi^{-1}(1-\alpha)$ = 0.84

Так как конверсию можно представить распределением Бернулли, то дисперсия - это произведение вероятности конверсии p и вероятности неудачи (1-p)

Имеем:
$$n > \dfrac{16*p*(100-p)}{MDE^2}$$

## Определение MDE и размера выборки перед тестом

In [ ]:
def get_table_sample_size(p, effects):
    results = []
    for eff in effects:
        sample_size = int(np.ceil(16 * p * (100-p) / (eff**2)))
        results.append(sample_size)
    
    df_results = pd.DataFrame()
    df_results['sample_size'] = results
    
    df_results.index = pd.MultiIndex(
        levels=[[f'{x}%' for x in effects]],
        codes=[np.arange(len(effects))],
        names=['effects']
    )
    
    return df_results

In [ ]:
effects = np.linspace(0.1, 2, 20)

In [ ]:
p = 10 #конверсия на исторических данных

In [ ]:
get_table_sample_size(p, effects)

## Относительный MDE и размер выборки

In [ ]:
def get_table_sample_size(p, effects):
    results = []
    for eff in effects:
        eff_abs = eff * p
        sample_size = int(np.ceil(16 * p * (100-p) / (eff_abs**2)))
        results.append(sample_size)
    
    df_results = pd.DataFrame()
    df_results['sample_size'] = results
    
    df_results.index = pd.MultiIndex(
        levels=[[f'{x}%' for x in effects]],
        codes=[np.arange(len(effects))],
        names=['effects']
    )
    
    return df_results

In [ ]:
effects = np.linspace(0.01, 0.05, 5)
effects

In [ ]:
p = 10

In [ ]:
get_table_sample_size(p, effects)

## Расчёт p-value

In [ ]:
a_group_visits = 10000
a_group_conversions = 500
b_group_visits = 10100
b_group_conversions = 605

a_group_conv = a_group_conversions/a_group_visits
b_group_conv = b_group_conversions/b_group_visits

In [ ]:
print(f'Конверсия в первой группе: {round(a_group_conv*100, 2)}%, конверсия во второй группе: {round(b_group_conv*100, 2)}%')

In [ ]:
ir = round((b_group_conv / a_group_conv - 1) * 100, 2)
print(f'Относительное изменение: {ir} %')

Так как данных много и наблюдений независимы, можно использовать Z-тест

In [ ]:
def z_test(a_conv: float, b_conv: float, size_a: int, size_b: int) -> float:
    
    sigma = np.sqrt(a_conv*(1-a_conv)/size_a + b_conv*(1-b_conv)/size_b)
    Z = abs(b_conv-a_conv)/sigma
    
    return norm.sf(Z, scale=1, loc=0), Z

In [ ]:
p_value, Z = z_test(a_group_conv, b_group_conv, a_group_visits, b_group_visits)
print('P-value :', round(p_value, 4))
print('Z:', Z)

In [ ]:
def another_z_test(a_conv: float, b_conv: float, size_a: int, size_b: int) -> float:
    
    p = (a_conv*size_a + b_conv*size_b)/(size_a+size_b)
    
    sigma = np.sqrt(p*(1-p)*(1/size_a + 1/size_b))
    Z = abs(b_conv-a_conv)/sigma
    
    return norm.sf(Z, scale=1, loc=0)

In [ ]:
print('P-value:', round(another_z_test(a_group_conv, b_group_conv, a_group_visits, b_group_visits), 4))

## Продолжительность теста

In [ ]:
from datetime import date

start_date = date(2024, 1, 29)
end_date = date(2024, 2, 26)
delta = end_date - start_date
print(delta.days)